In [1]:
import sys
import os

import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('../src/'))

from utils import TextDataset
from models import Net
from trainner import Trainner

In [ ]:
subjects=['biology', 'cooking', 'crypto', 'diy', 'robotics', 'travel']
path2data = '../data/'

In [ ]:
dsets = {
    'train': TextDataset(subjects, path2data + 'train/', col_lst = ['tokens', 'subject'] , is_train = True),
    'valid': TextDataset(subjects, path2data + 'train/', col_lst = ['tokens', 'subject'], is_valid=True),
    'test':  TextDataset(subjects, path2data + 'test/', col_lst = ['tokens', 'subject'], is_test=True),
}

In [ ]:
df = dsets['train'].data

In [ ]:
df.head()

In [ ]:
Net = Net(1000, 10)
print(Net)

In [ ]:
trainner = Trainner()